In [5]:
import geopandas as gpd
import numpy
import rioxarray
import xarray

from geocube.api.core import make_geocube

In [8]:
!wget https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/IMG/USGS_NED_13_n42w091_IMG.zip


--2021-07-01 21:38:15--  https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/IMG/USGS_NED_13_n42w091_IMG.zip
Resolving prd-tnm.s3.amazonaws.com (prd-tnm.s3.amazonaws.com)... 52.92.163.225
Connecting to prd-tnm.s3.amazonaws.com (prd-tnm.s3.amazonaws.com)|52.92.163.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346901225 (331M) [application/x-zip]
Saving to: ‘USGS_NED_13_n42w091_IMG.zip.1’

USGS_NED_13_n42w091 100%[===================>] 330.83M  15.5MB/s    in 23s     

2021-07-01 21:38:39 (14.4 MB/s) - ‘USGS_NED_13_n42w091_IMG.zip.1’ saved [346901225/346901225]



In [9]:
elevation = rioxarray.open_rasterio(
    "zip://USGS_NED_13_n42w091_IMG.zip/USGS_NED_13_n42w091_IMG.img"
).squeeze().drop("band")
elevation.name = "elevation"

In [10]:
elevation

<xarray.DataArray 'elevation' (y: 10812, x: 10812)>
[116899344 values with dtype=float32]
Coordinates:
  * x            (x) float64 -91.0 -91.0 -91.0 -91.0 ... -90.0 -90.0 -90.0 -90.0
  * y            (y) float64 42.0 42.0 42.0 42.0 42.0 ... 41.0 41.0 41.0 41.0
    spatial_ref  int64 0
Attributes: (12/16)
    LAYER_TYPE:                 athematic
    RepresentationType:         ATHEMATIC
    STATISTICS_COVARIANCES:     518.857106997209
    STATISTICS_EXCLUDEDVALUES:  
    STATISTICS_MAXIMUM:         284.1125793457
    STATISTICS_MEAN:            213.62414075776
    ...                         ...
    STATISTICS_SKIPFACTORY:     1
    STATISTICS_STDDEV:          22.778435130562
    _FillValue:                 -3.4028234663853e+38
    scale_factor:               1.0
    add_offset:                 0.0
    long_name:                  Layer_1

In [15]:
ssurgo_data = gpd.read_file("/content/data/soil_data_group.geojson")
ssurgo_data = ssurgo_data.loc[ssurgo_data.hzdept_r==0]
# convert the key to group to the vector data to an integer as that is one of the
# best data types for this type of mapping. If your data is not integer,
# then consider using a mapping of your data to an integer with something
# like a categorical dtype.
ssurgo_data["mukey"] = ssurgo_data.mukey.astype(int)

In [18]:
out_grid = make_geocube(
    vector_data=ssurgo_data,
    measurements=["mukey"],
    like=elevation, # ensure the data are on the same grid
    fill=numpy.nan
)


In [19]:
out_grid

<xarray.Dataset>
Dimensions:      (x: 10812, y: 10812)
Coordinates:
  * y            (y) float64 42.0 42.0 42.0 42.0 42.0 ... 41.0 41.0 41.0 41.0
  * x            (x) float64 -91.0 -91.0 -91.0 -91.0 ... -90.0 -90.0 -90.0 -90.0
    spatial_ref  int64 0
Data variables:
    mukey        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [20]:
out_grid["elevation"] = elevation
out_grid

<xarray.Dataset>
Dimensions:      (x: 10812, y: 10812)
Coordinates:
  * x            (x) float64 -91.0 -91.0 -91.0 -91.0 ... -90.0 -90.0 -90.0 -90.0
  * y            (y) float64 42.0 42.0 42.0 42.0 42.0 ... 41.0 41.0 41.0 41.0
    spatial_ref  int64 0
Data variables:
    mukey        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    elevation    (y, x) float32 221.3 221.2 221.2 221.2 ... 213.1 213.1 213.1

In [ ]:
grouped_elevation = out_grid.drop("spatial_ref").groupby(out_grid.mukey)
grid_mean = grouped_elevation.mean().rename({"elevation": "elevation_mean"})
grid_min = grouped_elevation.min().rename({"elevation": "elevation_min"})
grid_max = grouped_elevation.max().rename({"elevation": "elevation_max"})
grid_std = grouped_elevation.std().rename({"elevation": "elevation_std"})

In [ ]:
zonal_stats = xarray.merge([grid_mean, grid_min, grid_max, grid_std])
zonal_stats.to_dataframe()